In [25]:
import pandas as pd
import numpy as np

import scipy
import statsmodels.formula.api as smf
import statistics as st 
import pandas as pd
import matplotlib.pyplot as plt

import requests
from geopy.geocoders import Nominatim
from astral import LocationInfo
import datetime
from astral.sun import sun
from astral import LocationInfo

from time import time

In [26]:
cd

/Users/karatechop


In [27]:
cd /Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022\ \(CSV\)

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022 (CSV)


In [28]:
working_main = pd.read_csv('working_main.csv')
print(working_main.columns,'\n',working_main.shape,'\n',len(working_main.time))

/var/folders/td/yc8lq2qx2ks3ck292tbf6wtm0000gn/T/ipykernel_49295/2978794120.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  working_main = pd.read_csv('working_main.csv')


Index(['date', 'time', 'sex', 'race', 'type', 'duration', 'city', 'location'], dtype='object') 
 (2408246, 8) 
 2408246


In [29]:
print(working_main.columns,'\n\n',
      working_main.time.value_counts(),'\n\n',
      'unique lengths of timestamps are:',pd.unique([len(i) for i in working_main['time']]),'\n\n',
      [i for i in working_main['time'] if len(i)==5][0],'\n',
      [i for i in working_main['time'] if len(i)==8][0],'\n',
      [i for i in working_main['time'] if len(i)==7][0],'\n',
      [i for i in working_main['time'] if len(i)==19][0],'\n',
      [i for i in working_main['time'] if len(i)==4][0],'\n')

Index(['date', 'time', 'sex', 'race', 'type', 'duration', 'city', 'location'], dtype='object') 

 09:00       6871
10:00       6659
08:30       6604
09:30       6010
08:00       5933
            ... 
06:29:20       1
06:49:43       1
15:01:54       1
06:54:21       1
06:04:55       1
Name: time, Length: 83446, dtype: int64 

 unique lengths of timestamps are: [ 5  8  7 19  4] 

 13:00 
 07:58:50 
 8:18:00 
 1900-01-01 01:56:00 
 0:02 



In [30]:
time = []

for i in working_main['time']: 
    if len(i) == 8:
        time.append(i)
    elif len(i) == 4:
        time.append('0' + i + ':00')
    elif len(i) == 5:
        time.append(i + ':00')
    elif len(i) == 7:
        time.append('0' + i)
    elif len(i) == 19:
        time.append(i[-8:])
    else:
        time.append(i)

In [31]:
len(time)

2408246

In [32]:
working_main['time'] = time

# working_main['time'] = [pd.to_datetime(i) for i in working_main['time'] if len(i)==8]working_main['time'] = [pd.to_datetime(i) for i in working_main['time'] if len(i)==8]
working_main['time'] = pd.to_datetime(working_main['time'],format="%H:%M:%S")

print('len of time column is:',len(working_main.time),'\n sum of null time values is:',working_main['time'].isna().sum())
working_main

len of time column is: 2408246 
 sum of null time values is: 0


,date,time,sex,race,type,duration,city,location
0,12/09/2021,1900-01-01 13:00:00,Male,hispanic/latino(a),Non-Moving,5.0,ALL-SJPD-AB953,0 block of n jackson av
1,04/26/21,1900-01-01 23:00:00,Male,black/african american,Moving,60.0,ALL-SJPD-AB953,0 block of north second st
2,04/15/21,1900-01-01 11:35:00,Female,black/african american,Equipment,10.0,ALL-SJPD-AB953,100 block of graham ave
3,04/01/21,1900-01-01 01:53:00,Male,asian,Non-Moving,100.0,ALL-SJPD-AB953,100 block of on ramp/tully rd
4,12/20/2021,1900-01-01 16:07:00,Female,white,Moving,10.0,ALL-SJPD-AB953,100 block of sb/ mckee
...,...,...,...,...,...,...,...,...
2408241,12/31/21,1900-01-01 22:33:00,Male,black/african american,Non-Moving,10.0,ALL-SFPD-AB953,VIA BUFANO / COLUMBUS AVE
2408242,12/31/21,1900-01-01 20:33:00,Male,black/african american,Equipment,11.0,ALL-SFPD-AB953,HAHN ST / VISITACION AVE
2408243,12/31/21,1900-01-01 11:51:00,Male,group 4,Moving,27.0,ALL-SFPD-AB953,S. McDonnell Road at TNC Lot 1 SFO
2408244,12/31/21,1900-01-01 12:47:00,Male,black/african american,Moving,1252.0,ALL-SFPD-AB953,Northgate / Macallan Rd


In [33]:
working_main.rename(columns={'city':'agency'},inplace=True)
print('debug, change worked')

def city_from_agency_check(df):
    city_str = []
    lat_str = []
    lon_str = []
    wave_str = []
    
    for i in df['agency']:
        if 'ALL-SJPD-AB953' in i:
            city_str.append('San Jose')
            lat_str.append(37.3387)
            lon_str.append(-121.8853)
            wave_str.append(2)
        elif 'ALL-LBPD-AB953' in i:
            city_str.append('Long Beach')
            lat_str.append(33.7701)
            lon_str.append(-118.1937)
            wave_str.append(2)
        elif 'ALL-OAKLAND_PD-AB953' in i:
            city_str.append('Oakland')
            lat_str.append(37.8044)
            lon_str.append(-122.2712)
            wave_str.append(2)
        elif 'Berkeley PD' in i:
            city_str.append('Berkeley')
            lat_str.append(37.8715)
            lon_str.append(-122.2730)
            wave_str.append(3)
        elif 'ALL-MARIN-AB953' in i:
            city_str.append('Marin')
            lat_str.append(38.0834)
            lon_str.append(-122.7633)
            wave_str.append(3)
        elif 'Davis PD' in i:
            city_str.append('Davis')
            lat_str.append(38.5449)
            lon_str.append(-121.7405)
            wave_str.append(3)
        elif 'ALL-RIVERSIDE-AB953' in i:
            city_str.append('Riverside')
            lat_str.append(33.9806)
            lon_str.append(-117.3755)
            wave_str.append(1)
        elif 'SDPD' in i:
            city_str.append('San Diego')
            lat_str.append(32.7157)
            lon_str.append(-117.1611)
            wave_str.append(1)
        elif 'ALL-SFPD-AB953' in i:
            city_str.append('San Francisco')
            lat_str.append(37.7749)
            lon_str.append(-122.4194)
            wave_str.append(1)
        elif 'LAPD' in i:
            city_str.append('Los Angeles')
            lat_str.append(34.0522)
            lon_str.append(-118.2437)
            wave_str.append(1)
        else:
            print('didnt work',i)
        
    df['city']=city_str
    df['lat']=lat_str
    df['lon']=lon_str
    df['wave']=wave_str
    
    pd.option_context('display.precision', 10)
    
    return df

working_main = city_from_agency_check(working_main)
working_main['location'] = working_main['location'].astype(str)
working_main['location'] = working_main['location'] + ' - ' + working_main['city']

debug, change worked


In [34]:
working_main.location.value_counts()

nan - San Diego                               50516
nan - Long Beach                              17948
800 N Alameda St - Los Angeles                 5569
Roscoe Blvd & Nagle Ave - Los Angeles          4759
3000.0 - San Diego                             4718
                                              ...  
WASHINGTON & BURLINGTON - Los Angeles             1
N SPRiNG & 8TH - Los Angeles                      1
N MAIN & 9TH - Los Angeles                        1
500 W 1ST St - Los Angeles                        1
Northgate  /   Macallan Rd - San Francisco        1
Name: location, Length: 318586, dtype: int64

In [35]:
# working_main.to_csv('working_main_expanded.csv')

In [36]:
# working_main_expanded = pd.read_csv('working_main_expanded.csv')

In [37]:
working_main_expanded = working_main

In [38]:
working_main_expanded.date = pd.to_datetime(working_main_expanded.date)

In [39]:
working_main_expanded.dtypes

date                object
time        datetime64[ns]
sex                 object
race                object
type                object
duration           float64
agency              object
location            object
city                object
lat                float64
lon                float64
wave                 int64
dtype: object

In [40]:
# s = sun(loc.observer, date=datetime.date(2021, 1, 15), tzinfo=loc.timezone)
# for key in ['dawn', 'dusk', 'noon', 'sunrise', 'sunset']:
#     print(f'{key:10s}:', s[key])

In [42]:
# %%timeit
# start = time()

sunset_list = []
dusk_list = []
counter = 0



for i in working_main_expanded['date']:
    year = i.year
    month = i.month
    day = i.day

    name = working_main_expanded['city'][counter]
    lat = working_main_expanded['lat'][counter]
    lon = working_main_expanded['lon'][counter]

    loc = LocationInfo(name=name, region='CA, USA', timezone='America/Los_Angeles', #https://en.wikipedia.org/wiki/List_of_tz_database_time_zones
                       latitude=lat, longitude=lon)
    


    s = sun(loc.observer, date=datetime.date(year, month, day), tzinfo=loc.timezone)
    sunset_list.append(s['sunset'].time())
    dusk_list.append(s['dusk'].time())
    
    counter+=1
    if counter%100000==0:
        print(f'debug, counter is at {counter}')
    
    
    
sunset_list2 = [str(i)[:8] for i in sunset_list]  
dusk_list2 = [str(i)[:8] for i in dusk_list]  

working_main_expanded['sunsettime'] = sunset_list2
working_main_expanded['dusktime'] = dusk_list2
print('debug, done!')
# print(f'that whole process took {(time()-start)/60} minutes')
# sunset_list

debug, counter is at 100000
debug, counter is at 200000
debug, counter is at 300000
debug, counter is at 400000
debug, counter is at 500000
debug, counter is at 600000
debug, counter is at 700000
debug, counter is at 800000
debug, counter is at 900000
debug, counter is at 1000000
debug, counter is at 1100000
debug, counter is at 1200000
debug, counter is at 1300000
debug, counter is at 1400000
debug, counter is at 1500000
debug, counter is at 1600000
debug, counter is at 1700000
debug, counter is at 1800000
debug, counter is at 1900000
debug, counter is at 2000000
debug, counter is at 2100000
debug, counter is at 2200000
debug, counter is at 2300000
debug, counter is at 2400000
debug, done!


In [68]:
import warnings
warnings.filterwarnings("ignore") 
date_to_subtract = datetime.datetime(1900, 1, 1)



working_main_expanded['mins_sunset'] = [round(((datetime.datetime.strptime(i, '%H:%M:%S') - date_to_subtract).total_seconds() / 60.0),0) 
                                           for i in sunset_list2]
print('debug, mins_sunset done')


working_main_expanded['mins_dusk'] = [round(((datetime.datetime.strptime(i, '%H:%M:%S') - date_to_subtract).total_seconds() / 60.0),0) 
                                           for i in dusk_list2]
print('debug, mins_dusk done')


working_main_expanded['mins_stop_time'] = [round(((i - date_to_subtract).total_seconds()/60),0) 
                                           for i in working_main_expanded['time']]
print('debug, mins_stop done')



working_main_expanded['mins_after_dusk'] = working_main_expanded['mins_stop_time'] - working_main_expanded['mins_dusk']
working_main_expanded['day_in_year'] = [i.day for i in working_main_expanded['date']]

working_main_expanded['dark'] = 0
working_main_expanded.loc[working_main_expanded['mins_after_dusk']>=0,'dark']=1

working_main_expanded.head(3)

debug, mins_sunset done
debug, mins_dusk done
debug, mins_stop done


,date,day_in_year,time,sunsettime,dusktime,sex,race,type,duration,agency,location,city,lat,lon,wave,mins_stop_time,mins_sunset,mins_dusk,mins_after_dusk,dark
0,2021-12-09 00:00:00,9,1900-01-01 13:00:00,16:49:31,17:19:07,Male,hispanic/latino(a),Non-Moving,5.0,ALL-SJPD-AB953,0 block of n jackson av - San Jose,San Jose,37.3387,-121.8853,2,780.0,1010.0,1039.0,-259.0,0
1,2021-04-26 00:00:00,26,1900-01-01 23:00:00,19:52:49,20:21:06,Male,black/african american,Moving,60.0,ALL-SJPD-AB953,0 block of north second st - San Jose,San Jose,37.3387,-121.8853,2,1380.0,1193.0,1221.0,159.0,1
2,2021-04-15 00:00:00,15,1900-01-01 11:35:00,19:42:53,20:10:24,Female,black/african american,Equipment,10.0,ALL-SJPD-AB953,100 block of graham ave - San Jose,San Jose,37.3387,-121.8853,2,695.0,1183.0,1210.0,-515.0,0


In [69]:
working_main_expanded = working_main_expanded[['date','day_in_year', 'time', 'sunsettime', 'dusktime', 'sex', 'race', 'type', 'duration', 'agency', 'location',
       'city', 'lat', 'lon', 'wave', 'mins_stop_time','mins_sunset','mins_dusk', 'mins_after_dusk','dark']]

In [70]:
working_main_expanded.to_csv('working_main_expanded.csv',index=False)

In [71]:
working_main_expanded.location.shape[0]

2408246